# Batch Active Learning

While it is possible to perform rudimentary data selection simply by randomly choosing samples, the batch of data thus drawn might not be the most informative one.
Choosing those samples whith the largest prediction uncertainties from trajectories often results in the selection of configurations from subsequent time steps.

Batch selection methods can be constructed to select informative and diverse data, with or without following the underlying distribution.

We will illustrate this in a mock learning on the fly setup.

In [ ]:
# TODO Model Training
# TODO run MD
# TODO illustrate selection vs max uncertainty